In [61]:
print("Ok")

Ok


In [62]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.document_loaders import PyPDFLoader, DirectoryLoader


In [63]:
PINECONE_API_KEY = "pcsk_36Xv9K_A4djaiHKUaQmUxDsJpQYgVxnGyC3Mc1zW9WXqMwX484TtqUsEv1f6MVrLSCci9w"
PINECONE_API_ENV = "us-east-1-aws"

In [64]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [65]:
extracted_data=load_pdf_file(data='data/')


In [66]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [67]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 665


In [68]:
#text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()


C:\Users\Admin\AppData\Local\Temp\ipykernel_20040\3158813755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Admin\anaconda3\envs\tony\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Admin\anaconda3\envs\tony\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_

In [69]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [70]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
#query_result

In [71]:
from pinecone import Pinecone

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define your index
index_name = "medical-chatbot"

# Connect to an existing index
index = pc.Index(index_name)
print(f"Connected to Pinecone index: {index_name}")


Connected to Pinecone index: medical-chatbot


In [79]:
from langchain.embeddings import HuggingFaceEmbeddings

# Download the embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

embedded_data = [
    {
        "id": str(i),  # Unique ID for each vector
        "values": embeddings.embed_query(chunk.page_content),
        "metadata": {"text": chunk.page_content}  # Use "text" instead of "content"
    }
    for i, chunk in enumerate(text_chunks)
]


In [82]:
#embedded_data

In [81]:
# Upsert data to Pinecone index
namespace = "medical-data"  # Optional namespace to organize data

index.upsert(vectors=embedded_data, namespace=namespace)
print(f"Uploaded {len(embedded_data)} vectors to namespace '{namespace}' in index '{index_name}'.")

Uploaded 665 vectors to namespace 'medical-data' in index 'medical-chatbot'.


In [83]:
index_name

'medical-chatbot'

In [84]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings

# Use the same embeddings object
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Reconnect to the Pinecone index
docsearch = Pinecone.from_existing_index(index_name="medical-chatbot", embedding=embeddings)


In [92]:
docsearch

In [87]:
for vector in embedded_data[:5]:  # Print the first few vectors for verification
    print(vector)

{'id': '0', 'values': [0.04848267510533333, -0.04890904948115349, 0.06018342450261116, 0.017780698835849762, -0.05528058484196663, 0.05329025909304619, -0.025006510317325592, -0.0012130122631788254, -0.1217874065041542, -0.017582274973392487, 0.06656841188669205, 0.04719303548336029, -0.025439325720071793, 0.020503757521510124, 0.05970555171370506, -0.006304287351667881, 0.08660849928855896, -0.01707780361175537, 0.04588714987039566, 0.028162574395537376, 0.08028795570135117, 0.016808124259114265, 0.09914903342723846, 0.05664315074682236, -0.04959980770945549, 0.023187128826975822, 0.002655615098774433, 0.03656485676765442, -0.03582809865474701, 0.01622864231467247, 0.01246054656803608, -0.03233269974589348, 0.06859848648309708, 0.025024788454174995, -0.05546011030673981, 0.019654877483844757, 0.1450340747833252, -0.045986879616975784, -0.10847331583499908, 0.03157274052500725, -0.0763157531619072, -0.0666714683175087, 0.02901940420269966, 0.04943273216485977, 0.011303487233817577, 0.0

In [91]:
for doc in docs:
    print(doc.metadata.get('text', 'No text found'))

In [93]:
query = "How much physical activity do adults need each week?"
docs = docsearch.similarity_search(query, k=3, namespace="medical-data")

# Debugging: Print raw document data
print("Retrieved Documents:")
for doc in docs:
    print(doc)

Retrieved Documents:
page_content='300 minutes (5 hours) of moderate-intensity physical activity a week.
•  Adults should also do muscle-strengthening activities of moderate or greater intensity and that
involve all major muscle groups on 2 or more days a week, as these activities provide additional
health benefits.'
page_content='For people who are insufficiently active, that is, people who do some moderate- or vigorous-intensity 
physical activity, but who do not yet meet the key guidelines target range (150 to 300 minutes a week of 
moderate-intensity physical activity for adults): 
  • Even small increases in moderate-intensity physical activity provide health benefits. There is no 
threshold that must be exceeded before benefits begin to occur.'
page_content='minutes to 5 hours) of moderate-intensity aerobic physical activity each week. Adults gain additional and more 
extensive health benefits with even more physical activity. Muscle-strengthening activities also provide health 


In [46]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [47]:
# Create a PromptTemplate object
PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Step 3: Create an LLMChain
chain_type_kwargs = {"prompt" : PROMPT}

In [49]:
# Initialize LLM with CTransformers
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",  # Path to the Llama 2 model file
    model_type="llama",  # Specify model type
    config={
        "max_new_tokens": 512,  # Maximum tokens to generate
        "temperature": 0.8      # Sampling temperature
    }
)

In [94]:
retriever = docsearch.as_retriever(search_kwargs={"k": 3, "namespace": "medical-data"})


In [95]:
# Initialize the RetrievalQA object
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,  # Your initialized Llama 2 model
    retriever=retriever,  # The retriever from docsearch
    return_source_documents=True,  # Optional: include source documents in the output
    chain_type="stuff",  # Default chain type for combining retrieved data
    chain_type_kwargs={"prompt": PROMPT},  # Custom prompt template
)

In [96]:
query = "How much physical activity do adults need each week?"
response = retrieval_qa({"query": query})

# Display the response
print("Answer:", response['result'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_20040\537432137.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = retrieval_qa({"query": query})


Answer: Adults should aim to get at least 150 minutes of moderate-intensity aerobic physical activity or 75 minutes of vigorous-intensity aerobic physical activity per week, or an equivalent combination of both. Additionally, adults should do muscle-strengthening activities at least two times per week.
